In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import requests, datetime
import bs4
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import pandas as pd
import numpy as np
from GoogleNews import GoogleNews
from serpapi import GoogleSearch
from yahoo_fin import stock_info as si
import requests, json
from datetime import date
import calendar

In [ ]:
##Access to emails
##Enter personal email for the bot and recepients
sender_account = "" #bot email
sender_username = ""  #bot username
sender_password = ""#bot password
smtp_server = "" #server email for the bot, smtp.gmail.com if a gmail account
smtp_port = 587 #port number mainly associated for gmail bots
recepients = ["","",""] #recepients

In [ ]:
##Google News
news_url="https://news.google.com/news/rss"
Client=urlopen(news_url)
xml_page=Client.read()
Client.close()

daily_news=pd.DataFrame(columns=["Title","Link","Date Published"])

soup_page=soup(xml_page,"xml")
news_list=soup_page.findAll("item")
# Print news title, url and publish date
for news in news_list[0:10]:
  title=news.title.text
  link=news.link.text
  datepub=news.pubDate.text
  daily_news=daily_news.append(pd.DataFrame({'Title':title,'Link': link, 'Date Published': datepub},index=[0]), ignore_index=True)

from pretty_html_table import build_table
DNoutput = build_table(daily_news, 'blue_light') ##Format table of outputs

In [ ]:
##Google News for only Marvel 
googlenews = GoogleNews()
googlenews = GoogleNews(lang='en')
googlenews = GoogleNews(encode='utf-8')
currenttime = datetime.datetime.now()
googlenews = GoogleNews(start=currenttime,end=currenttime) #Establishes only today's news
googlenews.get_news('Marvel') #Enter keyword

marvel_news=pd.DataFrame(columns=['Title','Link','Date Published'])

for x in range(len(googlenews.get_texts()[0:10])):
    marvel_news=marvel_news.append(pd.DataFrame({'Title': googlenews.get_texts()[x], 'Link': googlenews.get_links()[x], 'Date Published': currenttime }, index=[0]),ignore_index=True)

from pretty_html_table import build_table
marvel_newseoutput = build_table(marvel_news, 'blue_light')

In [ ]:
##Weather Ops
##Weather information taken from OpenWeather API
 

api_key = "" #API Key
 

base_url = "http://api.openweathermap.org/data/2.5/weather?" #API URL

weather_forecast=pd.DataFrame(columns=["City","Temperature","Humidity","Description"]) 


city_name = "" #Enter desired city
 

complete_url = base_url + "appid=" + api_key + "&q=" + city_name

response = requests.get(complete_url)

x = response.json()

if x["coord"] != "404":

    y = x["main"]
    current_temperature = y["temp"]
    far_temp=round(((float(current_temperature) - 273.15) * 9/5 + 32)) #Conversion to fahrenheit from kelvin
    current_humidity = y["humidity"]
    z = x["weather"]
    weather_description = z[0]["description"]
    weather_forecast=weather_forecast.append(pd.DataFrame({'City':city_name,'Temperature': far_temp, 'Humidity': current_humidity, 'Description': weather_description},index=[0]), ignore_index=True)

else:
    print(" City Not Found ")

from pretty_html_table import build_table
Weatheroutput = build_table(weather_forecast, 'blue_light')

In [ ]:
##Premire League
result = requests.get("https://www.bbc.co.uk/sport/football/tables") ##Online resource https://www.sportperformanceanalysis.com/article/collecting-sports-data-web-scraping
src = result.content
soup = soup(src, 'html.parser')
table = soup.find_all("table")
league_table = table[0]
teams = league_table.find_all("tr")
PLTable=pd.DataFrame(columns=["Position","Team","Played","Won","Loss","Draw","Goal Diff","Points"])
for team in teams[1:21]: #Extract stats 

    stats = team.find_all("td")

    position = stats[0].text
    team_name = stats[2].text
    played = stats[3].text
    won = stats[4].text
    drawn = stats[5].text
    lost = stats[6].text
    goal_diff = stats[9].text
    points = stats[10].text
    PLTable=PLTable.append(pd.DataFrame({'Position':position,'Team': team_name, 'Played': played, 'Won':won,'Loss': lost, 'Draw': drawn, 'Goal Diff': goal_diff, 'Points': points},index=[0]), ignore_index=True)

from pretty_html_table import build_table
PLoutput = build_table(PLTable, 'blue_light')

In [ ]:
##NBA Scores
##API in use is SerpAPI
params = {
  "q": "NBA Scores",
  "api_key": "" #Use your API Key
} ##Queries google search

search = GoogleSearch(params)
results = search.get_dict()
sports_results = results['sports_results']

nba_Scores=[]
for i in range(len(sports_results['games'])):
    for x in range(len(sports_results['games'][i]['teams'])):
        #print(sports_results['games'][i]['teams'][x]['name'])
        #print(sports_results['games'][i]['teams'][x]['score'])
        nba_Scores.append(sports_results['games'][i]['teams'][x]['name'])
        nba_Scores.append(sports_results['games'][i]['teams'][x]['score'])


NBAScores=pd.DataFrame(columns=['Away','Score-A','Home','Score-H'])
i=0
while i < (len(nba_Scores)):
    NBAScores=NBAScores.append(pd.DataFrame({'Away': nba_Scores[i], 'Score-A':nba_Scores[i+1], 'Home': nba_Scores[i+2], 'Score-H': nba_Scores[i+3]}, index=[0]),ignore_index=True)
    i=i+4

from pretty_html_table import build_table
NBAScoresoutput = build_table(NBAScores, 'blue_light')    

In [ ]:
##NHL Scores
##API in use is SerpAPI
params = {
  "q": "NHL Scores",
  "api_key": "" #API Key
}

search = GoogleSearch(params)
results = search.get_dict()
sports_results = results['sports_results']

NHL_Scores=[]
for i in range(len(sports_results['games'])):
    for x in range(len(sports_results['games'][i]['teams'])):
        #print(sports_results['games'][i]['teams'][x]['name'])
        #print(sports_results['games'][i]['teams'][x]['score'])
        NHL_Scores.append(sports_results['games'][i]['teams'][x]['name'])
        NHL_Scores.append(sports_results['games'][i]['teams'][x]['score'])


NHLScores=pd.DataFrame(columns=['Away','Score-A','Home','Score-H'])
i=0
while i < (len(NHL_Scores)):
    NHLScores=NHLScores.append(pd.DataFrame({'Away': NHL_Scores[i], 'Score-A':NHL_Scores[i+1], 'Home': NHL_Scores[i+2], 'Score-H': NHL_Scores[i+3]}, index=[0]),ignore_index=True)
    i=i+4

from pretty_html_table import build_table
NHLScoresoutput = build_table(NHLScores, 'blue_light')  

In [ ]:
##NFL Scores
##API in use is SerpAPI
params = {
  "q": "NFL Scores",

  "api_key": "" #API Key
}

search = GoogleSearch(params)
results = search.get_dict()
sports_results = results['sports_results']

nfl_Scores=[]
for i in range(len(sports_results['games'])):
    for x in range(len(sports_results['games'][i]['teams'])):
        #print(sports_results['games'][i]['teams'][x]['name'])
        #print(sports_results['games'][i]['teams'][x]['score'])
        nfl_Scores.append(sports_results['games'][i]['teams'][x]['name'])
        nfl_Scores.append(sports_results['games'][i]['teams'][x]['score'])


NFLScores=pd.DataFrame(columns=['Away','Score-A','Home','Score-H'])
i=0
while i < (len(nfl_Scores)):
    NFLScores=NFLScores.append(pd.DataFrame({'Away': nfl_Scores[i], 'Score-A':nfl_Scores[i+1], 'Home': nfl_Scores[i+2], 'Score-H': nfl_Scores[i+3]}, index=[0]),ignore_index=True)
    i=i+4

from pretty_html_table import build_table
NFLScoresoutput = build_table(NFLScores, 'blue_light')    

In [ ]:
##Stocks

#Identify stocks of direct interest (Tech stocks here for example)
stock = [['Apple', si.get_live_price("aapl")], ['Amazon', si.get_live_price("amzn")], ['Tesla', si.get_live_price("tsla")], ['Facebook', si.get_live_price("fb")], ['Microsoft', si.get_live_price("msft")], ['ARKK', si.get_live_price("arkk")]]
stocks = pd.DataFrame(stock, columns = ['Ticker', 'Price'])

#Daily most active stocks
stocks_most_active = pd.DataFrame(si.get_day_most_active())

from pretty_html_table import build_table
TechStockoutput = build_table(stocks, 'blue_light')

from pretty_html_table import build_table
StocksMostActiveoutput = build_table(stocks_most_active, 'blue_light')

In [ ]:
##Function to build email body
#Function taken from and modified from GitHub  https://gist.github.com/JoeThunyathep/7b5020100ab15bf5399a5cc51836baec#file-sendcovidemail-py 
def Email (DNoutput,marvel_newseoutput,PLoutput,TechStockoutput,StocksMostActiveoutput,Weatheroutput,NBAScoresoutput,NFLScoresoutput,NHLScoresoutput,time):
    my_date = date.today()
    day=calendar.day_name[my_date.weekday()]
    email_subject = f"My Daily Update at {time}"
    email_body = '<html><head></head><body>'
    email_body += '<style type="text/css"></style>' 
    email_body += f'<h2>Good Morning, Here Is Your Daily Briefing</h2>' 

    #Daily News

    email_body += f'<b>Top News</b>: ' 
    email_body += f'{DNoutput}</h1>'

    email_body += f'<b>Top Marvel News</b>: ' 
    email_body += f'{marvel_newseoutput}</h1>'

    ##Weather
    email_body += f'<b>New York City Weather</b>: ' 
    email_body += f'{Weatheroutput}</h1>'
    
    ##Stocks
    email_body += f'<b>Tech Stocks</b>: ' 
    email_body += f'{TechStockoutput}</h1>'
    email_body += f'<b>Most Active Stocks</b>: ' 
    email_body += f'{StocksMostActiveoutput}</h1>'

    ##Sports
    email_body += f'<b>NBA Scores</b>: ' 
    email_body += f'{NBAScoresoutput}</h1>'

    email_body += f'<b>NHL Scores</b>: ' 
    email_body += f'{NHLScoresoutput}</h1>'
    if day=='Monday':
        

        email_body += f'<b>NFL Scores</b>: ' 
        email_body += f'{NFLScoresoutput}</h1>'
    else:
        email_body += f'<b>New NFL Scores Released on Mondays, Previous Week Scores</b>:'
        email_body += f'{NFLScoresoutput}</h1>'

    #Premier League

    email_body += f'<b> \r\r\n </b>' 
    email_body += f'<b>Current Premier League Standings</b>: ' 
    email_body += f'{PLoutput}</h1>'

    #footer
    email_body += '<br>Powered By' 
    email_body += '<br>Ben Bot</body></html>'

    server = smtplib.SMTP(smtp_server,smtp_port) 
    print(f"Logging in to {sender_account}")
    server.starttls() 
    server.login(sender_username, sender_password)
    for recipient in recepients:
        print(f"Sending email to {recipient}")
        message = MIMEMultipart('alternative') 
        message['From'] = sender_account 
        message['To'] = recipient 
        message['Subject'] = email_subject 
        message.attach(MIMEText(email_body, 'html')) 
        server.sendmail(sender_account,recipient,message.as_string())
    server.quit()

In [ ]:
Time = datetime.datetime.now()
Email(DNoutput,marvel_newseoutput,PLoutput,TechStockoutput,StocksMostActiveoutput,Weatheroutput,NBAScoresoutput,NFLScoresoutput,NHLScoresoutput,Time)